# All-sky Radiative Transfer Example with PyRTE-RRTMGP

This notebook demonstrates how to use the PyRTE-RRTMGP package to solve an idealized problem including clouds and clear skies. PyRTE-RRTMGP is a Python implementation of the Radiative Transfer for Energetics (RTE).

## Overview

PyRTE-RRTMGP provides a flexible and efficient framework for computing radiative fluxes in planetary atmospheres. This example shows an end-to-end problem with both clear skies and clouds. 

1. Loading data for cloud and gas optics 
2. Computing gas and cloud optical properties and combining them to produce an all-sky problem
3. Solving the radiative transfer equation to obtain upward and downward fluxes
4. Validating results against reference solutions generated with the original RTE fortran code

The package leverages `xarray` to represent data.

## Key Components

- **Gas and Cloud Optics**: Handles spectral properties of atmospheric gases and clouds and combines them to make a complete problem
- **RTE Solver**: Computes radiative fluxes based on atmospheric properties

This example demonstrates the workflow for both longwave and shortwave radiative transfer calculations.

See the [documentation](https://pyrte-rrtmgp.readthedocs.io/en/latest/) for more information.

## Preliminaries

### Installing dependencies

In [1]:
import numpy as np

### Importing pyRTE components

In [2]:
from pyrte_rrtmgp import rrtmgp_cloud_optics, rrtmgp_gas_optics
from pyrte_rrtmgp.data_types import (
    AllSkyExampleFiles,
    CloudOpticsFiles,
    GasOpticsFiles,
    OpticsProblemTypes,
)
from pyrte_rrtmgp.rte_solver import rte_solve
from pyrte_rrtmgp.utils import (
    compute_clouds,
    compute_profiles,
    load_rrtmgp_file,
)

### Setting up the problem

The routine `compute_profiles()` packaged with `pyRTE_RRTMGP` computes temperature, pressure, and humidity profiles following a moist adibat. The concentrations of other gases are also needed. Clouds are distributed in 2/3 of the columns 

In [3]:
def make_profiles(ncol=24, nlay=72):
    # Create atmospheric profiles and gas concentrations
    atmosphere = compute_profiles(300, ncol, nlay)

    # Add other gas values
    gas_values = {
        "co2": 348e-6,
        "ch4": 1650e-9,
        "n2o": 306e-9,
        "n2": 0.7808,
        "o2": 0.2095,
        "co": 0.0,
    }

    for gas_name, value in gas_values.items():
        atmosphere[gas_name] = value

    return atmosphere


atmosphere = make_profiles()
atmosphere

<xarray.Dataset> Size: 84kB
Dimensions:              (site: 24, layer: 72, level: 73)
Dimensions without coordinates: site, layer, level
Data variables: (12/13)
    pres_layer           (site, layer) float64 14kB 9.883e+04 ... 1.213
    temp_layer           (site, layer) float64 14kB 298.6 295.9 ... 200.6 200.6
    pres_level           (site, level) float64 14kB 1.015e+05 ... 1.065
    temp_level           (site, level) float64 14kB 300.0 297.2 ... 200.6 200.6
    surface_temperature  (site) int64 192B 300 300 300 300 ... 300 300 300 300
    h2o                  (site, layer) float64 14kB 0.01768 0.01583 ... 1e-08
    ...                   ...
    co2                  float64 8B 0.000348
    ch4                  float64 8B 1.65e-06
    n2o                  float64 8B 3.06e-07
    n2                   float64 8B 0.7808
    o2                   float64 8B 0.2095
    co                   float64 8B 0.0
Attributes:
    description:  Atmospheric profiles following RCEMIP protocol
    sst:          300
    ncol:         24
    nlay:         72

## Longwave calculations

In this example datasets are saved to intermediate variables at each step

### Initialize the cloud and gas optics data 

In [4]:
cloud_optics_lw = rrtmgp_cloud_optics.load_cloud_optics(
    cloud_optics_file=CloudOpticsFiles.LW_BND
)
gas_optics_lw = rrtmgp_gas_optics.load_gas_optics(
    gas_optics_file=GasOpticsFiles.LW_G256
)

cloud_optics_lw, gas_optics_lw

(<xarray.Dataset> Size: 14kB
 Dimensions:                (nrghice: 3, nband: 16, nsize_ice: 18,
                             nsize_liq: 20, pair: 2)
 Dimensions without coordinates: nrghice, nband, nsize_ice, nsize_liq, pair
 Data variables:
     asyice                 (nrghice, nband, nsize_ice) float32 3kB 0.413 ... ...
     asyliq                 (nband, nsize_liq) float32 1kB 0.04438 ... 0.9175
     extice                 (nrghice, nband, nsize_ice) float32 3kB 0.0749 ......
     extliq                 (nband, nsize_liq) float32 1kB 0.0705 ... 0.07606
     bnd_limits_wavenumber  (nband, pair) float32 128B 10.0 250.0 ... 3.25e+03
     ssaice                 (nrghice, nband, nsize_ice) float32 3kB 0.214 ... ...
     ssaliq                 (nband, nsize_liq) float32 1kB 0.02628 ... 0.6133
     diamice_lwr            float32 4B 10.0
     diamice_upr            float32 4B 180.0
     radliq_lwr             float32 4B 2.5
     radliq_upr             float32 4B 21.5
 Attributes:
     NCO: 

### Atmospheric profiles - clear sky, then clouds

In [5]:
atmosphere = make_profiles()
#
# Temporary workaround - compute_clouds() needs to know the particle size;
#   that's set as the mid-point of the valid range from cloud_optics
#
cloud_props = compute_clouds(
    cloud_optics_lw, atmosphere["pres_layer"], atmosphere["temp_layer"]
)

atmosphere = atmosphere.merge(cloud_props)
atmosphere

<xarray.Dataset> Size: 139kB
Dimensions:              (site: 24, layer: 72, level: 73)
Dimensions without coordinates: site, layer, level
Data variables: (12/17)
    pres_layer           (site, layer) float64 14kB 9.883e+04 ... 1.213
    temp_layer           (site, layer) float64 14kB 298.6 295.9 ... 200.6 200.6
    pres_level           (site, level) float64 14kB 1.015e+05 ... 1.065
    temp_level           (site, level) float64 14kB 300.0 297.2 ... 200.6 200.6
    surface_temperature  (site) int64 192B 300 300 300 300 ... 300 300 300 300
    h2o                  (site, layer) float64 14kB 0.01768 0.01583 ... 1e-08
    ...                   ...
    o2                   float64 8B 0.2095
    co                   float64 8B 0.0
    lwp                  (site, layer) float64 14kB 0.0 0.0 10.0 ... 0.0 0.0 0.0
    iwp                  (site, layer) float64 14kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    rel                  (site, layer) float64 14kB 0.0 0.0 12.0 ... 0.0 0.0 0.0
    rei                  (site, layer) float64 14kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    description:  Atmospheric profiles following RCEMIP protocol
    sst:          300
    ncol:         24
    nlay:         72

### Clear-sky (gases) optical properties; surface boundary conditions 

In [6]:
optical_props = gas_optics_lw.compute_gas_optics(
    atmosphere, problem_type=OpticsProblemTypes.ABSORPTION, add_to_input=False
)
optical_props["surface_emissivity"] = 0.98
optical_props

<xarray.Dataset> Size: 11MB
Dimensions:                  (site: 24, gpt: 256, layer: 72, level: 73,
                              bnd: 16, pair: 2)
Coordinates:
  * site                     (site) int64 192B 0 1 2 3 4 5 ... 18 19 20 21 22 23
  * gpt                      (gpt) int64 2kB 0 1 2 3 4 5 ... 251 252 253 254 255
Dimensions without coordinates: layer, level, bnd, pair
Data variables:
    surface_source           (gpt, site) float64 49kB 0.6522 ... 3.006e-06
    layer_source             (layer, gpt, site) float64 4MB 0.6477 ... 4.661e-09
    level_source             (level, gpt, site) float64 4MB 0.6522 ... 4.661e-09
    surface_source_jacobian  (gpt, site) float64 49kB 0.003262 ... 1.407e-07
    tau                      (site, layer, gpt) float64 4MB 40.04 ... 0.002916
    surface_emissivity       float64 8B 0.98
    bnd_limits_gpt           (bnd, pair) int32 128B 1 16 17 32 ... 240 241 256
Attributes:
    problem_type:     Longwave absorption
    top_at_1:         False
    dataset_mapping:  {"dim_mapping": {"site": "site", "layer": "layer", "lev...

### Calculate cloud properties; create all-sky optical properties

First compute the optical properties of the clouds

In [7]:
clouds_optical_props = cloud_optics_lw.compute_cloud_optics(
    atmosphere, problem_type=OpticsProblemTypes.ABSORPTION
)
# The optical properties of the clouds alone
clouds_optical_props

<xarray.Dataset> Size: 221kB
Dimensions:  (site: 24, layer: 72, bnd: 16)
Coordinates:
  * site     (site) int64 192B 0 1 2 3 4 5 6 7 8 ... 15 16 17 18 19 20 21 22 23
Dimensions without coordinates: layer, bnd
Data variables:
    tau      (layer, bnd, site) float64 221kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    dataset_mapping:  {"dim_mapping": {"site": "site", "layer": "layer", "lev...

Then add the optical properties of the clouds to the clear sky to get the total

In [8]:
# add_to() changes the value of `optical_props`
clouds_optical_props.add_to(optical_props)
optical_props

<xarray.Dataset> Size: 11MB
Dimensions:                  (site: 24, gpt: 256, layer: 72, level: 73,
                              bnd: 16, pair: 2)
Coordinates:
  * site                     (site) int64 192B 0 1 2 3 4 5 ... 18 19 20 21 22 23
  * gpt                      (gpt) int64 2kB 0 1 2 3 4 5 ... 251 252 253 254 255
Dimensions without coordinates: layer, level, bnd, pair
Data variables:
    surface_source           (gpt, site) float64 49kB 0.6522 ... 3.006e-06
    layer_source             (layer, gpt, site) float64 4MB 0.6477 ... 4.661e-09
    level_source             (level, gpt, site) float64 4MB 0.6522 ... 4.661e-09
    surface_source_jacobian  (gpt, site) float64 49kB 0.003262 ... 1.407e-07
    tau                      (site, layer, gpt) float64 4MB 40.04 ... 0.002916
    surface_emissivity       float64 8B 0.98
    bnd_limits_gpt           (bnd, pair) int32 128B 1 16 17 32 ... 240 241 256
Attributes:
    problem_type:     Longwave absorption
    top_at_1:         False
    dataset_mapping:  {"dim_mapping": {"site": "site", "layer": "layer", "lev...

### Compute broadband fluxes

In [9]:
fluxes = rte_solve(optical_props, add_to_input=False)
fluxes

<xarray.Dataset> Size: 42kB
Dimensions:              (site: 24, level: 73)
Coordinates:
  * site                 (site) int64 192B 0 1 2 3 4 5 6 ... 18 19 20 21 22 23
Dimensions without coordinates: level
Data variables:
    lw_flux_up_jacobian  (site, level) float64 14kB nan nan nan ... nan nan nan
    lw_flux_up           (site, level) float64 14kB 458.9 449.9 ... 277.4 277.4
    lw_flux_down         (site, level) float64 14kB 441.7 429.1 ... 0.01084 0.0

### Compare to reference results

In [10]:
# Load reference data and verify results
ref_data = load_rrtmgp_file(AllSkyExampleFiles.LW_NO_AEROSOL)
assert np.isclose(
    fluxes["lw_flux_up"],
    ref_data["lw_flux_up"].T,
    atol=1e-7,
).all()

assert np.isclose(
    fluxes["lw_flux_down"],
    ref_data["lw_flux_dn"].T,
    atol=1e-7,
).all()

print("All Checks Passed!")

All Checks Passed!


# Shortwave calculations

In this example steps are combined where possible

## Initialize optics data 

In [11]:
cloud_optics_sw = rrtmgp_cloud_optics.load_cloud_optics(
    cloud_optics_file=CloudOpticsFiles.SW_BND
)
gas_optics_sw = rrtmgp_gas_optics.load_gas_optics(
    gas_optics_file=GasOpticsFiles.SW_G224
)

cloud_optics_sw, gas_optics_sw

(<xarray.Dataset> Size: 13kB
 Dimensions:                (nrghice: 3, nband: 14, nsize_ice: 18,
                             nsize_liq: 20, pair: 2)
 Dimensions without coordinates: nrghice, nband, nsize_ice, nsize_liq, pair
 Data variables:
     asyice                 (nrghice, nband, nsize_ice) float32 3kB 0.8563 ......
     asyliq                 (nband, nsize_liq) float32 1kB 0.7908 ... 0.8657
     extice                 (nrghice, nband, nsize_ice) float32 3kB 0.3251 ......
     extliq                 (nband, nsize_liq) float32 1kB 0.4448 ... 0.07082
     bnd_limits_wavenumber  (nband, pair) float32 112B 820.0 2.68e+03 ... 5e+04
     ssaice                 (nrghice, nband, nsize_ice) float32 3kB 0.7774 ......
     ssaliq                 (nband, nsize_liq) float32 1kB 0.8496 ... 0.9999
     diamice_lwr            float32 4B 10.0
     diamice_upr            float32 4B 180.0
     radliq_lwr             float32 4B 2.5
     radliq_upr             float32 4B 21.5
 Attributes:
     NCO:  

### Atmospheric profiles - clear sky, then clouds

In [12]:
atmosphere = make_profiles()
#
# Temporary workaround - compute_clouds() needs to know the particle size;
#    that's set as the mid-point of the valid range from cloud_optics
#
atmosphere = atmosphere.merge(
    compute_clouds(
        cloud_optics_sw, atmosphere["pres_layer"], atmosphere["temp_layer"]
    )
)
atmosphere

<xarray.Dataset> Size: 139kB
Dimensions:              (site: 24, layer: 72, level: 73)
Dimensions without coordinates: site, layer, level
Data variables: (12/17)
    pres_layer           (site, layer) float64 14kB 9.883e+04 ... 1.213
    temp_layer           (site, layer) float64 14kB 298.6 295.9 ... 200.6 200.6
    pres_level           (site, level) float64 14kB 1.015e+05 ... 1.065
    temp_level           (site, level) float64 14kB 300.0 297.2 ... 200.6 200.6
    surface_temperature  (site) int64 192B 300 300 300 300 ... 300 300 300 300
    h2o                  (site, layer) float64 14kB 0.01768 0.01583 ... 1e-08
    ...                   ...
    o2                   float64 8B 0.2095
    co                   float64 8B 0.0
    lwp                  (site, layer) float64 14kB 0.0 0.0 10.0 ... 0.0 0.0 0.0
    iwp                  (site, layer) float64 14kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    rel                  (site, layer) float64 14kB 0.0 0.0 12.0 ... 0.0 0.0 0.0
    rei                  (site, layer) float64 14kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    description:  Atmospheric profiles following RCEMIP protocol
    sst:          300
    ncol:         24
    nlay:         72

### Compute gas and cloud optics and combine in one step

In [13]:
# compute_cloud_optics() returns two-stream properties by default?
atmosphere["surface_albedo"] = 0.06
atmosphere["surface_albedo_direct"] = 0.06
atmosphere["surface_albedo_diffuse"] = 0.06
optical_props = gas_optics_sw.compute_gas_optics(
    atmosphere, problem_type=OpticsProblemTypes.TWO_STREAM, add_to_input=False
)
# add_to() changes the values in optical_props
cloud_optics_sw.compute_cloud_optics(atmosphere).add_to(
    optical_props, delta_scale=True
)
#
# Add SW-specific surface and angle properties
#
optical_props["surface_albedo"] = 0.06
optical_props["surface_albedo_direct"] = 0.06
optical_props["surface_albedo_diffuse"] = 0.06
# Could also specific a single "surface_albedo"
optical_props["mu0"] = 0.86
optical_props

<xarray.Dataset> Size: 9MB
Dimensions:                 (site: 24, gpt: 224, layer: 72, bnd: 14, pair: 2)
Coordinates:
  * site                    (site) int64 192B 0 1 2 3 4 5 ... 18 19 20 21 22 23
Dimensions without coordinates: gpt, layer, bnd, pair
Data variables:
    toa_source              (site, gpt) float64 43kB 1.994 1.938 ... 0.001233
    tau                     (site, layer, gpt) float64 3MB 0.02873 ... 0.0001194
    ssa                     (site, layer, gpt) float64 3MB 5.438e-05 ... 0.1774
    g                       (site, layer, gpt) float64 3MB 0.0 0.0 ... 0.0 0.0
    surface_albedo_direct   float64 8B 0.06
    surface_albedo_diffuse  float64 8B 0.06
    bnd_limits_gpt          (bnd, pair) int32 112B 1 16 17 32 ... 208 209 224
    surface_albedo          float64 8B 0.06
    mu0                     float64 8B 0.86
Attributes:
    problem_type:     Shortwave 2-stream
    top_at_1:         False
    dataset_mapping:  {"dim_mapping": {"site": "site", "layer": "layer", "lev...

### Compute fluxes

In [14]:
fluxes = rte_solve(optical_props, add_to_input=False)
fluxes

IndexError: Axis 1 of variable <this-array> is out of bounds of the expanded dimension size 1

### Compare to reference results

In [ ]:
ref_data = load_rrtmgp_file(AllSkyExampleFiles.SW_NO_AEROSOL)
assert np.isclose(
    fluxes["sw_flux_up"],
    ref_data["sw_flux_up"].T,
    atol=1e-7,
).all()
assert np.isclose(
    fluxes["sw_flux_down"],
    ref_data["sw_flux_dn"].T,
    atol=1e-7,
).all()
assert np.isclose(
    fluxes["sw_flux_dir"],
    ref_data["sw_flux_dir"].T,
    atol=1e-7,
).all()

print("All Checks Passed!")